# New Def

지난번 만든 함수에는 user_list를 만들어서 data가 일부 소실되는 바람에 submission 값도 소실되는 결과가 있었음.

따라서 이번에는 ID 중복을 drop하지 않는 방향으로 진행해 볼 것임

또한, occyp_type을 미리 처리해주고 ID에 occyp_type을 추가하는 시도를 해볼 것임.

+ 함수를 처리하는 중 print로 data를 확인하는 부분이 있는데, 이부분을 color 변경을 통해서 주목성을 높임
  

## 필요한 라이브러리 호출

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from sklearn.preprocessing import OrdinalEncoder


warnings.filterwarnings('ignore')

## 데이터 전처리 과정

In [195]:
# 경로 설정 (dataset을 저장하는 경로임.)
path = 'data/'

train = pd.read_csv('{}train.csv'.format(path))
test = pd.read_csv('{}test.csv'.format(path))

In [196]:
#쓸모없는 칼럼 삭제
for df in (train,test):
    df.drop(['FLAG_MOBIL'], axis=1, inplace=True)   
    df.drop(columns=['index'], inplace=True)
    
del df

### 1. 나이, 고용일자를 이용하여 파생변수 형성

In [197]:
# 1st

def year_month_total(df_1, df_2, column, column_name):
    """[나이, 고용연수 파생변수를 생성하기 위한 함수입니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        column ([생성할 컬럼의 기준 컬럼 명]): [생성할 컬럼의 기준 컬럼 명을 입력해주세요]
        column_name ([생성할 컬럼의 이름]): [연단위, 연+월단위로 표시된 파생변수 이름입니다.]
    """
    # 변수 설정
    
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']
    
    # 입력한 컬럼이 DAYS_BIRTH인 경우 If로 처리
    
    if (column == 'DAYS_BIRTH') == True:    
        for dataset,name in zip(df_list,names):
            dataset['{}_year'.format(column_name)] = dataset[column].abs()//365
            dataset['{}_month'.format(column_name)] = (dataset[column].abs()%365)//30
            dataset['{}_total'.format(column_name)] = dataset['{}_year'.format(column_name)] + round(dataset['{}_month'.format(column_name)] * (1/12), 2)
            dataset.drop(columns=['{}_month'.format(column_name)], inplace=True)
        
            print('{0}set에 {1}_year, {1}_total 컬럼을 생성하였습니다. \n'.format(name,column_name))
    
    # 입력한 컬럼이 DAYS_EMPLOYED인 경우 else로 처리 (occyp_type 처리를 위한 작업이므로 1번째로 수행)
    
    else:
        for dataset,name in zip(df_list,names):
            print('{}set에 근무일자가 잘못기입된 '.format(name) +
                  '\033[96m' + '{} cases'.format(len(dataset[dataset['DAYS_EMPLOYED']>=0])) + 
                  '\033[0m' + '에 대한 처리를 시작합니다.')
            dataset['DAYS_EMPLOYED'] = dataset['DAYS_EMPLOYED'].replace(365243, 0)
            
            dataset['{}_year'.format(column_name)] = dataset[column].abs()//365
            dataset['{}_month'.format(column_name)] = (dataset[column].abs()%365)//30
            dataset['{}_total'.format(column_name)] = dataset['{}_year'.format(column_name)] + round(dataset['{}_month'.format(column_name)] * (1/12), 2)
            dataset.drop(columns=['{}_month'.format(column_name)], inplace=True)
        
            print('{0}set에 {1}_year, {1}_total 컬럼을 생성하였습니다. \n'.format(name, column_name))
        
    del df_list,names
    return dataset.head(1)

    

In [198]:
year_month_total(train, test,'DAYS_BIRTH','age')

trainset에 age_year, age_total 컬럼을 생성하였습니다. 

testset에 age_year, age_total 컬럼을 생성하였습니다. 



,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,age_year,age_total
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,0,1,0,NaN,2.0,-60.0,60,60.25


In [199]:
year_month_total(train, test,'DAYS_EMPLOYED','work')

trainset에 근무일자가 잘못기입된 4438 cases에 대한 처리를 시작합니다.
trainset에 work_year, work_total 컬럼을 생성하였습니다. 

testset에 근무일자가 잘못기입된 1697 cases에 대한 처리를 시작합니다.
testset에 work_year, work_total 컬럼을 생성하였습니다. 



,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,work_phone,phone,email,occyp_type,family_size,begin_month,age_year,age_total,work_year,work_total
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,...,0,1,0,NaN,2.0,-60.0,60,60.25,0,0.0


### 2. 직업 Nan 값 처리

In [200]:
# 2nd

def occyp_type_nan(df_1, df_2):
    """[직업 컬럼 내의 Nan 값을 처리합니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
    """
    # 변수 설정
    
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    # 무직자 처리 함수
    
    for dataset,name in zip(df_list,names):
        dataset['occyp_type'] = dataset['occyp_type'].fillna('Nan')
        
        print('직업 컬럼의 ' +
              '\033[92m' + 'Nan value {}명 중 '.format(len(dataset.loc[dataset['occyp_type'] == 'Nan'])) +
              '\033[96m' + '{}명이 실제 고용일수가 0일입니다. '.format(len(dataset.loc[(dataset['occyp_type'] == 'Nan') & (dataset['DAYS_EMPLOYED'] == 0)])) + 
              '\033[0m'+ '따라서 jobless로 처리합니다.')
        
        dataset.loc[(dataset['occyp_type'] == 'Nan') & (dataset['DAYS_EMPLOYED'] == 0), 'occyp_type'] = 'jobless'
        
        # 무직자는 아니지만 데이터가 없는 경우
        
        print('나머지 {}명 데이터는 no_data로 처리합니다.'.format(len(dataset.loc[dataset['occyp_type'] == 'Nan'])))
        
        dataset.loc[dataset['occyp_type'] == 'Nan', 'occyp_type'] = 'no data'
        
        print('{}set의 직업 Nan value를 처리하였습니다.\n'.format(name))
        
    del df_list, names
    return dataset.head(1)

occyp_type_nan(train, test)

직업 컬럼의 Nan value 8171명 중 4438명이 실제 고용일수가 0일입니다. 따라서 jobless로 처리합니다.
나머지 3733명 데이터는 no_data로 처리합니다.
trainset의 직업 Nan value를 처리하였습니다.

직업 컬럼의 Nan value 3152명 중 1697명이 실제 고용일수가 0일입니다. 따라서 jobless로 처리합니다.
나머지 1455명 데이터는 no_data로 처리합니다.
testset의 직업 Nan value를 처리하였습니다.



,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,work_phone,phone,email,occyp_type,family_size,begin_month,age_year,age_total,work_year,work_total
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,...,0,1,0,jobless,2.0,-60.0,60,60.25,0,0.0


### 3. ID컬럼 생성

In [201]:
def ID_col(df_1, df_2):
    """[데이터 셋 전처리 함수입니다.]

    Args:
        df_1 ([Dataset]): [train 데이터셋을 입력해주세요]
        df_2 ([Dataset]): [test 데이터셋을 입력해주세요]
    """
    
    # 변수 설정
     
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    a = []
    change_credit = 0
    names = ['train', 'test']
    
    # ID 생성
    
    # for name, dataset, in zip(names, df_list):
    for dataset in df_list:
        dataset['ID'] = dataset['gender'].astype(str) +'.'+ dataset['car'].astype(str)+'.' + dataset['reality'].astype(str)+'.' + \
                dataset['child_num'].astype(str)+'.' + dataset['income_total'].astype(str) +'.'+ dataset['income_type'].astype(str) +'.'+ \
                dataset['edu_type'].astype(str)+'.' + dataset['family_type'].astype(str)+'.' + dataset['house_type'].astype(str) +'.'+ \
                dataset['DAYS_BIRTH'].astype(str) +'.'+ dataset['DAYS_EMPLOYED'].astype(str) +'.'+ dataset['work_phone'].astype(str) +'.'+ \
                dataset['phone'].astype(str) +'.'+ dataset['email'].astype(str) +'.' + dataset['family_size'].astype(str)+'.' + dataset['occyp_type'].astype(str)
        
        dataset = dataset.sort_values(['ID','begin_month'], ascending=[True, False])
        dataset.reset_index(inplace=True)
        dataset.drop(columns={'index'}, inplace=True)

        # OrdinalEncoder로 ID 변환
        
        print('\033[96m' + '해당 Data 내의 고유 ID 수는 {}입니다. '.format(dataset['ID'].nunique()) + 
              '\033[0m' + 'OrdinalEncoder를 이용힌 변환을 시작합니다.')
        encode = OrdinalEncoder()
        dataset[['ID']] = encode.fit_transform(dataset[['ID']])

        # Nth_card 컬럼 추가 (ID가 같은데 카드 개수가 여러 개인 사람을 대상으로 몇 번째 카드를 만들었는지 생성해준다)
        
        print('Column(Nth_card)을 생성합니다.')
        for i in tqdm(range(dataset['ID'].nunique())):
            Nth_card = len(dataset[dataset['ID']==i]) + 1
            
            while Nth_card > 0:
                Nth_card = Nth_card -1
                a.append(Nth_card)
                if Nth_card == 1:
                    break
                
        dataset['Nth_card'] = a        
        a.clear()
        print('\033[92m' + '신규로 카드를 개설한 사람의 수 : {}'.format(dataset[dataset['begin_month']==0.0]['ID'].nunique()) +
              '\033[0m')
        
        # Credit이 좋아지지 않고 악화된 경우를 체크하는 부분 (begin_month기준으로 Nth_card를 추가해줬으므로 인덱스 값을 기준으로 시행한다.)
        
        if (dataset['ID'].nunique() > 8000)==True:
            print('credit 정보를 확인하고 시간이 지나며 credit이 악화된 사례가 있는지 확인합니다.')
            for i in tqdm(range(dataset['ID'].nunique())):
                num = dataset.loc[dataset['ID'] == i]['credit'].to_list()
                for j in range(len(num)-1):
                    if (num[j] <= num[j+1]) == True:
                        pass
                    else :
                        change_credit += 1
                        break
            print('\033[96m'+'{}명 (전체 인원 중 {:.2f}%)은 credit이 개선되지 않고 악화된 사례가 존재합니다.'.format(change_credit, 
                                                                        (change_credit/dataset['ID'].nunique())*100) + 
                  '\033[0m')  
        else:
            print('\033[96m' + 'credit 정보가 없는 testset입니다.' + 
                  '\033[0m')
        
        # dataset 저장부분
        
        print('dataset(user_list_{})을 저장합니다. \n'.format(name))    
        dataset.to_csv('{}user_list_{}.csv'.format(path, name))
        
    
    del names, a
    return dataset.head(5)

In [202]:
ID_col(train, test)

해당 Data 내의 고유 ID 수는 8759입니다. OrdinalEncoder를 이용힌 변환을 시작합니다.
Column(Nth_card)을 생성합니다.


100%|██████████| 8759/8759 [00:05<00:00, 1633.31it/s]


신규로 카드를 개설한 사람의 수 : 214
credit 정보를 확인하고 시간이 지나며 credit이 악화된 사례가 있는지 확인합니다.


100%|██████████| 8759/8759 [00:06<00:00, 1413.73it/s]


1842명 (전체 인원 중 21.03%)은 credit이 개선되지 않고 악화된 사례가 존재합니다.
dataset(user_list_test)을 저장합니다. 

해당 Data 내의 고유 ID 수는 5585입니다. OrdinalEncoder를 이용힌 변환을 시작합니다.
Column(Nth_card)을 생성합니다.


100%|██████████| 5585/5585 [00:03<00:00, 1573.29it/s]


신규로 카드를 개설한 사람의 수 : 82
credit 정보가 없는 testset입니다.
dataset(user_list_test)을 저장합니다. 



,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,email,occyp_type,family_size,begin_month,age_year,age_total,work_year,work_total,ID,Nth_card
0,F,N,N,0,103500.0,Pensioner,Higher education,Married,House / apartment,-21701,...,0,jobless,2.0,-3.0,59,59.42,0,0.0,0.0,3
1,F,N,N,0,103500.0,Pensioner,Higher education,Married,House / apartment,-21701,...,0,jobless,2.0,-46.0,59,59.42,0,0.0,0.0,2
2,F,N,N,0,103500.0,Pensioner,Higher education,Married,House / apartment,-21701,...,0,jobless,2.0,-53.0,59,59.42,0,0.0,0.0,1
3,F,N,N,0,103500.0,Pensioner,Higher education,Single / not married,House / apartment,-24543,...,0,jobless,1.0,-28.0,67,67.17,0,0.0,1.0,1
4,F,N,N,0,103500.0,Pensioner,Secondary / secondary special,Married,House / apartment,-20123,...,0,jobless,2.0,-11.0,55,55.08,0,0.0,2.0,2


In [203]:
train = pd.read_csv('{}user_list_train.csv'.format(path))
test = pd.read_csv('{}user_list_test.csv'.format(path))

### faimly_size 조정

In [204]:
def replace_value_family(df_1, df_2, num_of_outlier):
    """[Family_size의 outlier를 대치해주는 작업을 시행합니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        num_of_outlier ([outlier_standard]): [family_size의 outlier 기준을 입력하세요]
    """
    # 변수 설정
    
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    # family_size Outlier를 처리하는 작업
    
    for dataset,name in zip(df_list,names):
        
        print('family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.')
        
        dataset.drop(columns=['child_num'], inplace=True)
        
        print('\033[96m' + 'family_size {}명 이상인 사람의 수 : {}'.format(num_of_outlier, 
                                                                      len(dataset.loc[dataset['family_size'] >= num_of_outlier])) +
              '\033[0m')
        
        sub = dataset.loc[dataset['family_size']>= num_of_outlier]
        
        for i in tqdm(range(len(sub))):
        
            a = int(dataset[dataset['family_type'] == sub['family_type'].values[i]].mean()['family_size'])
            
            dataset.loc[(dataset['family_size']>= num_of_outlier), 'family_size'] = a
        
        # 잘 처리되었는지 확인하는 부분
        
        print('\033[96m' + 'family_size {}명 이상인 사람의 수 : {}'.format(num_of_outlier, 
                                                        len(dataset.loc[dataset['family_size'] >= num_of_outlier]))+
              '\033[0m')

        # family_size를 조정하고 난 후에 1인당 소득으로 total_income을 scale_down 해줌.
        
        print('1인당 소득으로 소득 수준을 조정합니다.')
        
        dataset['income_mean'] = dataset['income_total'] / dataset['family_size']
        
        print('{} 데이터 셋 처리 완료.\n'.format(name))
    
    del sub,df_list, names
    return dataset.head(1)

replace_value_family(train, test, 7)

family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.
family_size 7명 이상인 사람의 수 : 15


100%|██████████| 15/15 [00:09<00:00,  1.60it/s]


family_size 7명 이상인 사람의 수 : 0
1인당 소득으로 소득 수준을 조정합니다.
train 데이터 셋 처리 완료.

family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.
family_size 7명 이상인 사람의 수 : 10


100%|██████████| 10/10 [00:01<00:00,  6.88it/s]

family_size 7명 이상인 사람의 수 : 0
1인당 소득으로 소득 수준을 조정합니다.
test 데이터 셋 처리 완료.



,Unnamed: 0,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,occyp_type,family_size,begin_month,age_year,age_total,work_year,work_total,ID,Nth_card,income_mean
0,0,F,N,N,103500.0,Pensioner,Higher education,Married,House / apartment,-21701,...,jobless,2.0,-3.0,59,59.42,0,0.0,0.0,3,51750.0


In [205]:
names = ['train', 'test']

for name,dataset in zip(names, [train, test]):
    dataset.drop(columns=['DAYS_BIRTH','DAYS_EMPLOYED'], inplace=True)
    dataset.to_csv('{}final_set_{}.csv'.format(path,name))